In [1]:
!pip3 install httpx==0.20 dash plotly
!python3 -m pip install packaging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 931.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 6.3 MB/s eta 0:00:00


In [2]:
import dash
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px


In [15]:
# Replace 'your_dataset.csv' with the path to your dataset
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

In [61]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas


# Ensure correct data types
data['Year'] = data['Year'].astype(int)
data['Recession'] = data['Recession'].astype(int)

# Initialize the Dash app
app = dash.Dash(__name__)

# List of years
year_list = data['Year'].unique()

# Create the layout of the app
app.layout = html.Div([
    html.H1("Automobile Statistics Dashboard", style={'color': '#503D36', 'textAlign': 'center'}),
    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'yearly'},
                {'label': 'Recession Period Statistics', 'value': 'recession'}
            ],
            value='yearly'
        )
    ]),
    html.Div([
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value=year_list.max() if year_list.size > 0 else None
        )
    ], style={'display': 'block'}),
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
])

# Callback to enable/disable year dropdown based on statistics type
@app.callback(
    Output('select-year', 'style'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown_style(statistics_type):
    if statistics_type == 'yearly':
        return {'display': 'block'}
    else:
        return {'display': 'none'}

# Callback for updating graphs
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'), Input('select-year', 'value')]
)
def update_output_container(statistics_type, selected_year):
    graphs = []

    if statistics_type == 'recession':
        recession_data = data[data['Recession'] == 1]

        # Recession Period Statistics Graphs
        # Graph 1: Yearly Automobile Sales
        yearly_sales = recession_data.groupby('Year')['Automobile_Sales'].sum().reset_index()
        graphs.append(dcc.Graph(
            figure=px.line(yearly_sales, x='Year', y='Automobile_Sales', title='Yearly Automobile Sales During Recession')
        ))

        # Graph 2: Total Monthly Automobile Sales
        monthly_sales = recession_data.groupby(['Year', 'Month'])['Automobile_Sales'].sum().reset_index()
        graphs.append(dcc.Graph(
            figure=px.line(monthly_sales, x='Month', y='Automobile_Sales', color='Year', title='Total Monthly Automobile Sales During Recession')
        ))

        # Graph 3: Average Vehicles Sold by Vehicle Type
        avg_vehicles_sold = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        graphs.append(dcc.Graph(
            figure=px.bar(avg_vehicles_sold, x='Vehicle_Type', y='Automobile_Sales', title='Average Vehicles Sold by Vehicle Type During Recession')
        ))

        # Graph 4: Total Advertisement Expenditure for Each Vehicle
        ad_exp = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        graphs.append(dcc.Graph(
            figure=px.pie(ad_exp, names='Vehicle_Type', values='Advertising_Expenditure', title='Total Advertisement Expenditure for Each Vehicle During Recession')
        ))

    elif statistics_type == 'yearly':
        yearly_data = data[data['Year'] == selected_year]

        # Yearly Statistics Graphs
        # Graph 1: Monthly Automobile Sales
        sales_by_month = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        graphs.append(dcc.Graph(
            figure=px.line(sales_by_month, x='Month', y='Automobile_Sales', title='Monthly Automobile Sales')
        ))

        # Graph 2: Average Number of Vehicles Sold by Vehicle Type
        avg_vehicles_sold_yearly = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        graphs.append(dcc.Graph(
            figure=px.bar(avg_vehicles_sold_yearly, x='Vehicle_Type', y='Automobile_Sales', title='Average Vehicles Sold by Vehicle Type')
        ))

        # Graph 3: Total Expenditure Share by Vehicle Type
        total_expenditure = yearly_data.groupby('Vehicle_Type').agg({'Advertising_Expenditure': 'sum', 'Price': 'sum'}).reset_index()
        total_expenditure['Total_Expenditure'] = total_expenditure['Advertising_Expenditure'] + total_expenditure['Price']
        graphs.append(dcc.Graph(
            figure=px.pie(total_expenditure, names='Vehicle_Type', values='Total_Expenditure', title='Total Expenditure Share by Vehicle Type')
        ))

        # Graph 4: Effect of Unemployment Rate on Vehicle Type and Sales
        unemployment_effect = yearly_data.groupby('Vehicle_Type').agg({'Automobile_Sales': 'mean', 'unemployment_rate': 'mean'}).reset_index()
        graphs.append(dcc.Graph(
            figure=px.bar(unemployment_effect, x='Vehicle_Type', y='Automobile_Sales', color='unemployment_rate', title='Effect of Unemployment Rate on Vehicle Type and Sales')
        ))

    return graphs

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>